In [1]:
import requests
import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import TwitterUtils as TU
import seaborn as sns
import re

# import spacy
# import spacy_langdetect as sld

In [2]:

import tweepy
import time

with open('my_oauth.json', 'r') as f:
    oauth_tokens = json.load(f)


consumer_key = os.environ.get("CONSUMER_KEY")
consumer_secret = os.environ.get("CONSUMER_SECRET")
access_token = oauth_tokens['oauth_token']
access_token_secret = oauth_tokens['oauth_token_secret']

# tweepy workflow, not sure how it differs from twint
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

## Combining Data

The `GetPlaces.py`, `GetTweets.py`, and `SampleUser.py` files have generated the following output files:
* users.json : contains all the user specific data from the 51,000 sampled users
* places.pkl : metadata related to all twitter places in the user sample
* tweets.pkl : retrieved 100 tweets from each user

In [3]:
URL_regex = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)'
twitter_username_re = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)'
twitter_username_re = r"((^|[^@\w])@(\w{1,15})\b)*"

In [13]:
with open("user_data_convo.pkl", "rb") as file:
    convos = pkl.load(file)

In [14]:
convo_df = pd.DataFrame(convos)

In [15]:
ids = convo_df.tweet_id.unique()

In [16]:
data_df = pd.read_csv('filtered_data.csv')

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,1,10,11,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
convo_df.columns

Index(['user_id', 'tweet_id', 'tweet_text', 'referenced_tweets', 'convo_id',
       'reply_to_user_id'],
      dtype='object')

In [35]:
data_df.tweet_id.dropna().unique()

array([1637850668214960128, 1637818231863001090, 1637737394911969280, ...,
       1637299644664549381, 1637040000335495170, 1636604644813328385],
      dtype=object)

In [41]:
data_df.tweet_id.astype(str)

0         1637850668214960128
1         1637818231863001090
2         1637737394911969280
3         1637733079002537986
4         1637660117624836096
                 ...         
944493    1637686649470730240
944494    1637307250380505093
944495    1637299644664549381
944496    1637040000335495170
944497    1636604644813328385
Name: tweet_id, Length: 944498, dtype: object

In [49]:
search_list = data_df.tweet_id.dropna().astype(str)

lengths = [len(tid) for tid in search_list]



search_list[lengths.index(16)]

'1.6338379328e+18'

In [50]:

original_ids = list(data_df.tweet_id.dropna().astype(np.int64))

ValueError: invalid literal for int() with base 10: 'df7fd3a3b9eff7ee'

In [51]:
# Collect specifically tweet ids

original_ids = list(data_df.tweet_id)
convo_ids = list(convo_df.tweet_id)
replied_to_list = [c['referenced_tweets'][-1].get('id') for c in convos]

tweet_id_set = set(replied_to_list)
tweet_id_set.update(original_ids)
tweet_id_set.update(convo_ids)


In [52]:
replied_to_list

['1637748357127864320',
 '1637749653906272256',
 '1637733079002537986',
 '1637748357127864320',
 '1638023870933344256',
 '1637850668214960128',
 '1637748357127864320',
 '1637748357127864320',
 '1637748357127864320',
 '1637749653906272256',
 '1637733079002537986',
 '1637748357127864320',
 '1638023870933344256',
 '1637850668214960128',
 '1637748357127864320',
 '1637748357127864320',
 '1637723095694815235',
 '1637891439681245190',
 '1639418704185139202',
 '1637723095694815235',
 '1637730989785989122',
 '1637723095694815235',
 '1637723095694815235',
 '1637723095694815235',
 '1637754165308035075',
 '1637723095694815235',
 '1637748357127864320',
 '1637749653906272256',
 '1637733079002537986',
 '1637748357127864320',
 '1638023870933344256',
 '1637850668214960128',
 '1637748357127864320',
 '1637748357127864320',
 '1637291464626679808',
 '1636556023887716352',
 '1636556023887716352',
 '1636556023887716352',
 '1636556023887716352',
 '1636556023887716352',
 '1636567213787480067',
 '16365560238877

In [54]:
id_list = list(ids)

In [55]:
id_list

['1638630321473335297',
 '1638112643293511681',
 '1638034126531104768',
 '1638031325126074369',
 '1638027677776113670',
 '1638023870933344256',
 '1638018184262606849',
 '1638003069635153921',
 '1640039436976287744',
 '1639832760180428800',
 '1639420281147080708',
 '1639418704185139202',
 '1639283766001532928',
 '1639282456846114816',
 '1638858997402058753',
 '1638848416762703872',
 '1638815869743824898',
 '1638708493996941314',
 '1637292033584033792',
 '1637291464626679808',
 '1636620772021096450',
 '1636616768092516352',
 '1636593371123978240',
 '1636570052752457728',
 '1636568248509665282',
 '1636567213787480067',
 '1636561629277044738',
 '1636557447627444226',
 '1637428071492886531',
 '1637334843997687809',
 '1637290149318119424',
 '1637284891569328128',
 '1637119964477014021',
 '1637119932822609921',
 '1637072344043896835',
 '1637061002662649859',
 '1636997687425007616',
 '1636923517852856321',
 '1636794470829273088',
 '1636753473629741056',
 '1636645108237078528',
 '16365998801385

In [11]:
out = api.get_status(id_list[0])

In [60]:
client = TU.TwitterClient()

{"data": [{"id": "1648485285930364928", "value": "(a OR the) has:geo lang:en tweets_count:50", "tag": "active_user"}], "meta": {"sent": "2023-04-19T00:40:56.426Z", "result_count": 1}}
{"meta": {"sent": "2023-04-19T00:40:56.669Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "(a OR the) has:geo lang:en tweets_count:50", "tag": "active_user", "id": "1648486807040847873"}], "meta": {"sent": "2023-04-19T00:40:58.114Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}


In [62]:
kernel_url = TU.BASE + 'tweets/'
kernel_params = {'tweet.fields' : 'author_id,conversation_id,created_at,in_reply_to_user_id,referenced_tweets'}
kernel_params['ids'] = ",".join(id_list)

output = client.connect_to_endpoint(kernel_url, kernel_params)

KeyboardInterrupt: 

In [1]:
type(id_list[0])

NameError: name 'id_list' is not defined

In [63]:
timestamps = []

for id_ in id_list[::100]:
    print(id_)
    kernel_params['ids'] = ",".join(id_)
   
    try:
        output = client.connect_to_endpoint(kernel_url, kernel_params)
        timestamps.append(output)
    except:
        continue

1638630321473335297
200
1636317706227007488
200
1638329723402035200
200
1635092597667684353
200
1636095582442799105
200
1634333838477344768
200
1637526157888303104
200
1638099666049933313
200
1641726878477066243
200
1638317617231413249
200
1637918339656736768
200
1637894289479946265
200
1636998625778876418
200
1638039992156016640
200
1641082510556950528
200
1641912990847889408
200
1636212008130420736
200
1634410134720880641
200
1636483582087557121
200
1643015116370092032
200
1637085318586449924
200
1640383887414030336
200
1636670685803315201
200
1636566021225185280
200
1637829621377900544
200
1636890852285898754
200
1634050531399282692
200
1637326887948099586
200
1637757130013831170
200
1642962182769082368
200
1636862892460113920
200
1642887729137549312
200
1634273901147504641
200
1637722488133017601
200
1638345194679459841
200
1637231550898085889
200
1636324941552398338
200
1635803803646586880
200
1636136282769309696
200
1636086028762062852
200
1637961427590717448
200
1633531462996615

In [1]:
timestamps

NameError: name 'timestamps' is not defined

431


Exception: (431, 'Request Header Fields Too Large')

In [17]:
len(output['data'])

99

In [ ]:
url = "https://api.twitter.com/2/users/{}/tweets".format(id)

In [15]:
30*196902 / 100 / 60 / 60

16.4085

In [13]:
len(timestamps)

99

In [14]:
timestamps

{'1638630321473335297': datetime.datetime(2023, 3, 22, 19, 54, 47, tzinfo=datetime.timezone.utc),
 '1638112643293511681': datetime.datetime(2023, 3, 21, 9, 37, 43, tzinfo=datetime.timezone.utc),
 '1638034126531104768': datetime.datetime(2023, 3, 21, 4, 25, 43, tzinfo=datetime.timezone.utc),
 '1638031325126074369': datetime.datetime(2023, 3, 21, 4, 14, 35, tzinfo=datetime.timezone.utc),
 '1638027677776113670': datetime.datetime(2023, 3, 21, 4, 0, 6, tzinfo=datetime.timezone.utc),
 '1638023870933344256': datetime.datetime(2023, 3, 21, 3, 44, 58, tzinfo=datetime.timezone.utc),
 '1638018184262606849': datetime.datetime(2023, 3, 21, 3, 22, 22, tzinfo=datetime.timezone.utc),
 '1638003069635153921': datetime.datetime(2023, 3, 21, 2, 22, 19, tzinfo=datetime.timezone.utc),
 '1640039436976287744': datetime.datetime(2023, 3, 26, 17, 14, 6, tzinfo=datetime.timezone.utc),
 '1639832760180428800': datetime.datetime(2023, 3, 26, 3, 32, 51, tzinfo=datetime.timezone.utc),
 '1639420281147080708': datetim

In [33]:
convos[0]

{'user_id': '1013303513449836547',
 'tweet_id': '1638630321473335297',
 'tweet_text': '@1shankarsharma @Pavanasoonu Think.... astrologers are more accurate than market pundits.',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1637748357127864320'}],
 'convo_id': '1637733079002537986',
 'reply_to_user_id': '2989553946'}

In [39]:
convo_dict = {}

for c in convos:
    if convo_dict.get(c['convo_id']):
        convo_dict[c['convo_id']].append(c)
    else:
        convo_dict[c['convo_id']] = [c]


In [53]:
replied_to_list = [c['referenced_tweets'][-1].get('id') for c in convos]

In [56]:
convo_df['referenced_tweets'] = replied_to_list

In [83]:
tweet_ids = convo_df.tweet_id.apply(str)
referenced_tweets = convo_df.referenced_tweets.apply(str)

In [65]:
og_data = pd.read_csv('filtered_data.csv')

/var/folders/b1/vw0sn5w90nnccbfn7bskv0jm0000gn/T/ipykernel_8359/3908887897.py:1: DtypeWarning: Columns (0,1,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  og_data = pd.read_csv('filtered_data.csv')


In [84]:
og_data['tweet_id'].dtype
convo_df['tweet_id'].dtype

dtype('O')

In [85]:
og_tweet_ids = og_data.tweet_id.apply(str)

In [82]:
og_tweet_ids

0         1637850668214960128
1         1637818231863001090
2         1637737394911969280
3         1637733079002537986
4         1637660117624836096
                 ...         
944493    1637686649470730240
944494    1637307250380505093
944495    1637299644664549381
944496    1637040000335495170
944497    1636604644813328385
Name: tweet_id, Length: 944498, dtype: object

In [88]:
any([ref in og_tweet_ids for ref in referenced_tweets])

False

In [16]:
filtered_convo = convo_df[convo_df['convo_id'] == convo_1]

In [26]:
filtered_convo.referenced_tweets.apply(lambda x: x[0].get('id'))

0         1637748357127864320
1         1637749653906272256
2         1637733079002537986
3         1637748357127864320
4         1638023870933344256
                 ...         
641677    1637748357127864320
641678    1638023870933344256
641679    1637850668214960128
641680    1637748357127864320
641681    1637748357127864320
Name: referenced_tweets, Length: 120, dtype: object

In [28]:
tweet_ids = filtered_convo.tweet_id.unique()

In [29]:
tweet_ids

array(['1638630321473335297', '1638112643293511681',
       '1638034126531104768', '1638031325126074369',
       '1638027677776113670', '1638023870933344256',
       '1638018184262606849', '1638003069635153921'], dtype=object)

In [32]:
places_unpacked = [item  for item in places.values()]
def unpack_place(place):
    return (place.id, place.name, place.full_name, place.country, place.country_code, place.place_type)

unpacked_places = [unpack_place(place) for place in places_unpacked]
place_df = pd.DataFrame(unpacked_places, columns = ("id", "name", "full_name", "country", "country_code", "type"))

URL_regex = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'

In [5]:
URL_regex = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)'

In [35]:
df = pd.DataFrame(data)
merged = pd.merge(df, place_df, how = 'left', left_on = 'place_id', right_on='id')
full_data = pd.merge(merged, users_df, how='left', left_on ='user_id', right_on = 'id')
# full_data.to_csv('fulldata.csv', index = False) # 3 Gigs of data, not great...
# full_data.description.iloc[0,]
# places_unpacked # Cool opportunity for geographic visualization

In [36]:
len(full_data['user_id'].unique())

30470

In [37]:
# Limit to top 6 countries
by_country = merged.groupby('country').count()
top6 = by_country.sort_values(by = 'user_id', ascending=False).head(6)
total = by_country.user_id.sum()
top6.user_id.divide(total).sum()

0.9066735266648764

In [38]:
full_data.columns

Index(['user_id', 'tweet_id', 'tweet_text', 'place_id', 'id_x', 'name',
       'full_name', 'country', 'country_code', 'type', 'username',
       'description', 'id_y', 'user_name_field', 'location', 'withheld'],
      dtype='object')

The top 6 countries account for 91% of the total users collected, which suggests pretty good coverage. Dropping unnecessary columns

Rename Columns

In [39]:
target_fields = ['user_id', 'tweet_id', 'tweet_text', 'place_id', 'name',
       'full_name', 'country', 'country_code', 'type', 'username',
       'description', 'user_name_field', 'location']
reduced_df = full_data[target_fields]
reduced_df = reduced_df.rename(columns={'name':'place_name', 
                                        'full_name':'full_place_name',
                                        'type': 'place_type', 
                                        'description':'profile_description',
                                        'user_name_field':'profile_name',
                                        'location':'profile_location'
                                        })

Drop Duplicates

In [40]:
top6_countries = top6.index
top6 = reduced_df[reduced_df['country'].isin(top6_countries)]
unique_tweets = top6['tweet_id'].unique()
top6 = top6.drop_duplicates('tweet_id')

Detect languages

In [41]:
def get_lang_detector(nlp, name):
    return sld.LanguageDetector()

# Uncomment when running for first time
nlp = spacy.load("en_core_web_sm")
spacy.Language.factory('language_detector', func = get_lang_detector)
nlp.add_pipe('language_detector', last =True)

def get_language(text):
    return nlp(text)._.language['score']

In [48]:
get_language(top6.tweet_text.iloc[0])

0.9999960942533268

In [49]:
data_dict = top6.to_dict(orient='records')
langs = [get_language(d['tweet_text']) for d in data_dict]

## Clean Tweet Texts

In [63]:
text_test = top6.tweet_text.iloc[1]
text_test

'@Gajendr70729189 @amitsharma2704 @1shankarsharma Including my SAP technology business.  Thank you. Namaste.'

In [66]:
re.sub(URL_regex, "", text_test)
re.sub(twitter_username_re, "", text_test)

' Including my SAP technology business.  Thank you. Namaste.'

In [42]:
def clean_text(text):
    temp = re.sub(URL_regex, "", text)

    return re.sub(twitter_username_re, "", text)

top6['clean_text'] = top6.tweet_text.apply(clean_text)

In [44]:
top6 = top6.drop_duplicates('tweet_id')
top6.shape

(859183, 14)

In [45]:
top6.to_csv('filtered_data.csv', index=False)